In [1]:
#we shall like to create a function which can go through several minimization algorithms for our data,
#the final goal being to find the refractive index of our material

In [2]:
from scipy import optimize
import 

In [ ]:
m = 
d = 
lam = 
phi_i = 

In [4]:
def F(m, d, lam, phi_i):
    
    def funk_to_optimize(n):
        return m - (2*d)/lam * (n*(1/math.cos(math.asin(math.sin(phi_i)/n)) - 1)
                                + 1 - (math.cos(phi_i - math.asin(math.sin(phi_i)/n))/math.cos(math.asin(math.sin(phi_i)/n))))